In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 2

import sys
sys.path.append('..')

from jax import numpy as jnp
import jax
import time
from scipy.integrate import quad, dblquad
import equinox as eqx
import jax_cosmo as jc
from quadax import quadgk, quadcc, quadts, romberg
from functools import partial

In [4]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.animation as animation 
from numpy import loadtxt
from diffrax import Kvaerno5, Tsit5
%matplotlib inline

from plot_params import params
#pylab.rcParams.update(params)

cols_default = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [5]:
from linx.background import BackgroundModel
from linx.nuclear import NuclearRates 
from linx.abundances import AbundanceModel
from linx.reactions import Reaction
from linx.thermo import T_g 
from linx.const import mn, mp, eta0, hbar, me, aFS
from scipy.integrate import odeint, solve_ivp, simpson, cumulative_simpson
from linx.pd_rates import decay_model
from linx.injected_spec import InjectedSpectrum
eps = 1e-3
Emin = 1.5 # MeV
Ephb_T_max = 200

In [6]:
bkg_model = BackgroundModel()
t_vec, a_vec, rho_g_vec, rho_nu_vec, rho_NP_vec, p_NP_vec, Neff_vec = bkg_model(jnp.asarray(0.), T_end = 1e-4, max_steps = 512*4)

`\         /´  ||||        ||||  |||||     ||||  ||||   ||||
 /\_______/\   ||||        ||||  |||||||   ||||   |||| ||||
 ) __` ´__ (   ||||        ||||  |||| |||| ||||    |||||||
/  `-|_|-´  \  ||||        ||||  ||||  |||| |||    ||||||| 
/   (_x_)   \  ||||||||||  ||||  ||||   |||||||   |||| ||||
  )  `-´  (    ||||||||||  ||||  ||||    ||||||  ||||   ||||
 
Compiling thermodynamics model...


/home/mkwok/miniconda3/envs/linx/lib/python3.13/site-packages/equinox/_jit.py:55: UserWarning: `diffrax.diffeqsolve(..., discrete_terminating_event=...)` is deprecated in favour of the more general `diffrax.diffeqsolve(..., event=...)` interface. This will be removed in some future version of Diffrax.
  out = fun(*args, **kwargs)


In [7]:
InjSpec = InjectedSpectrum()
E_grid = jnp.linspace(1.5, jnp.exp(3), 100)

In [8]:
mod = decay_model(60, 1e5, 5e-7, 1, 0, t_vec, a_vec, rho_g_vec)

In [26]:
mod.get_pdi_grids()

Rate time:  1.1275253295898438
Kernel time:  0.07466292381286621
Source time:  0.008898019790649414
Solve time:  7.62939453125e-05


ValueError: invalid shape in fixed-type tuple.

In [13]:
E_th = {
    "dgnp" :  2.224573,
    "tgnd" :  6.257248,
    "tgnpn" :  8.481821,
    "He3gpd" :  5.493485,
    "He3gnpp" :  7.718058,
    "He4gpt" : 19.813852,
    "He4gnHe3" : 20.577615,
    "He4gdd" : 23.846527,
    "He4gnpd" : 26.071100,
    "Li6gnpHe4" :  3.698892,
    "Li6gX" : 15.794685,
    "Li7gtHe4" :  2.467032,
    "Li7gnLi6" :  7.249962,
    "Li7gnnpHe4" : 10.948850,
    "Be7gHe3He4" :  1.586627,
    "Be7gpLi6" :  5.605794,
    "Be7gppnHe4" :  9.304680
}

In [14]:
Eth = jnp.array(list(E_th.values()))

In [11]:
E_grid = jnp.logspace(jnp.log(Emin), jnp.log(5), 5, base=jnp.e)
ii, jj = jnp.meshgrid(jnp.arange(3), E_grid, indexing='ij')

In [13]:
InjSpec.rate_x(ii.flatten(), jj.flatten(), jnp.full_like(jj.flatten(), 0.01))

Array([1.48929727e-17, 1.18140444e-15, 2.96468909e-14, 3.12209257e-13,
       1.72739024e-12, 2.84328170e-10, 2.78964054e-10, 2.66403186e-10,
       2.49423545e-10, 2.29960223e-10, 2.84328170e-10, 2.78964054e-10,
       2.66403186e-10, 2.49423545e-10, 2.29960223e-10], dtype=float64)

In [11]:
mod.pd_tables

Array([[1.14776247e-004, 1.14776247e-003, 1.14776247e-002],
       [1.00000000e-200, 1.00000000e-200, 6.19915177e-032],
       [1.00000000e-200, 1.00000000e-200, 7.88891194e-037],
       [1.00000000e-200, 1.00000000e-200, 3.06808952e-067],
       [1.00000000e-200, 1.00000000e-200, 1.18055298e-035],
       [1.00000000e-200, 1.00000000e-200, 1.08726449e-056],
       [1.00000000e-200, 1.00000000e-200, 1.00000000e-200],
       [1.00000000e-200, 1.00000000e-200, 1.00000000e-200],
       [1.00000000e-200, 1.00000000e-200, 1.00000000e-200],
       [1.00000000e-200, 1.00000000e-200, 1.00000000e-200],
       [1.00000000e-200, 1.00000000e-200, 9.72145141e-035],
       [1.00000000e-200, 1.00000000e-200, 1.00000000e-200],
       [1.00000000e-200, 1.00000000e-200, 1.13931006e-032],
       [1.00000000e-200, 1.00000000e-200, 1.22790384e-047],
       [1.00000000e-200, 1.00000000e-200, 1.00000000e-200],
       [1.00000000e-200, 1.00000000e-200, 6.53803418e-032],
       [1.00000000e-200, 1.00000000e-200

In [ ]:
dir(mod)

In [ ]:
InjSpec.get_spectrum(5, mod.get_source_0(), mod.get_source_cont(), 0.01)

In [ ]:
mod.get_pdi_grids()

In [ ]:
rates_i = mod.pdi_rates(0.01)
print(rates_i)

In [14]:
pdi_rates = jnp.array([[1.79507704e-180, 7.61656453e-183, 2.25062597e-184,
        1.81990493e-182, 5.23902999e-184, 1.00000000e-200,
        1.00000000e-200, 1.00000000e-200, 1.00000000e-200,
        6.22968129e-182, 2.01147832e-192, 6.30950360e-181,
        9.25721900e-184, 1.72201013e-186, 1.02890632e-180,
        5.85469164e-183, 6.08196967e-185],
       [3.32704835e-029, 6.88650382e-031, 3.72505336e-032,
        1.32219725e-030, 7.37756567e-032, 2.07791113e-037,
        5.42095241e-038, 3.89011243e-100, 5.87007179e-144,
        3.54311411e-030, 1.18925388e-035, 1.18740610e-029,
        1.13249292e-031, 8.19466752e-034, 1.44151709e-029,
        3.88415678e-031, 1.30078527e-032],
       [7.69015092e-032, 1.20633964e-034, 7.98129583e-036,
        2.11540830e-034, 1.76685100e-035, 1.00000000e-200,
        1.00000000e-200, 1.00000000e-200, 1.00000000e-200,
        5.94929745e-034, 1.00000000e-200, 1.30081174e-032,
        2.37792950e-035, 1.00000000e-200, 1.05423791e-031,
        6.06419952e-035, 8.96912718e-037]], dtype=float)

In [ ]:
mod.get_cross_sections(E_grid)

In [ ]:
T_range = mod.temperature_range()
print(T_range)
T_grid = jnp.linspace(T_range[0], T_range[1], 10)
T_grid

In [20]:
F_grid = InjSpec.get_spectrum(5, mod.get_source_0(), mod.get_source_cont(), 0.01)
print(F_grid)

Rate time:  6.131103038787842
Kernel time:  0.07419157028198242
Source time:  0.37545013427734375
Solve time:  1.0881428718566895
[[1.50000000e+000 2.02680023e+000 2.73861279e+000 3.70041402e+000
  5.00000000e+000]
 [1.03519555e-022 8.34928073e-025 1.76063104e-026 3.79300659e-028
  8.12898254e-030]
 [6.00021510e-030 3.07237428e-030 2.40752876e-030 2.32581517e-030
  4.34857814e-191]
 [6.00021510e-030 3.07237428e-030 2.40752876e-030 2.32581517e-030
  4.34857814e-191]]


In [23]:
def F_s(log_E):
    E = jnp.exp(log_E)
    return jnp.exp(jnp.interp(jnp.log(E), jnp.log(F_grid[0]), jnp.log(F_grid[1]))) * E * mod.get_cross_section(E)


In [24]:
jax.vmap(quadgk, in_axes=(None, 0, None, None, None, None))(F_s, [jnp.log(Eth), jnp.full_like(Eth, jnp.log(200))], (), False, 0, eps,)

(Array([[5.75661445e-32, 4.78724821e-34, 5.16871323e-34, 5.31877511e-34,
         7.75789297e-34, 9.12597606e-34, 1.00511838e-33, 1.51393789e-36,
         1.90427992e-34, 9.54197677e-34, 1.38048594e-33, 5.16411415e-33,
         2.56851054e-34, 4.63427310e-34, 5.20672394e-32, 2.02389968e-34,
         4.29810951e-34],
        [6.83287665e-34, 4.78724185e-34, 5.16870498e-34, 5.30384414e-34,
         7.75789065e-34, 9.12582387e-34, 1.00510652e-33, 1.51392561e-36,
         1.90428138e-34, 9.42720195e-34, 1.38048472e-33, 5.46619965e-36,
         2.56832392e-34, 4.63427205e-34, 0.00000000e+00, 2.01630418e-34,
         4.29811463e-34],
        [5.98362672e-34, 4.64651690e-34, 5.16870959e-34, 5.05770217e-34,
         7.75499214e-34, 9.12587406e-34, 1.00510741e-33, 1.51392583e-36,
         1.90428109e-34, 8.92435475e-34, 1.38048524e-33, 2.10948961e-37,
         2.55584849e-34, 4.63427338e-34, 0.00000000e+00, 1.96011139e-34,
         4.29811471e-34],
        [7.17648126e-34, 4.78724167e-34, 5.168

In [ ]:
a = 5
def cond_fun(state):
  # Define the condition for the loop to continue
  return state[0] < 10

def body_fun(state):
  # Define the operations to perform in each iteration
  return (state[0] + 1, state[1]**2)

init_val = 0  # Initial value for the loop variable

final_state = jax.lax.while_loop(cond_fun, body_fun, (0, 5))

print(final_state)  # Output: 10

In [26]:
S_0f, S_contf = mod.get_source_0(), mod.get_source_cont()

NE = jnp.array(jnp.log10(5/Emin)*150, int)
NE = jnp.maximum(NE, 150)
NE = 10

#number of species
N_X = 3

E_grid = jnp.logspace(jnp.log(Emin), jnp.log(5), NE, base=jnp.e)

# Generate the grid for the different species
X_grid = jnp.arange(N_X)

#rate (MeV)
R = jnp.array([[InjSpec.rate_x(X, E, 0.01) for E in E_grid] for X in X_grid])
#R = jnp.array([self.rate_x(X_grid, E, T) for E in E_grid])
ii, jj, kk, ll = jnp.meshgrid( X_grid, X_grid,E_grid, E_grid,indexing='ij')
k = jnp.stack([ll, kk, jj, ii], axis=-1)
k_true = jnp.array(k[:, :, :, :, 0] >= k[:, :, :, :, 1])
K = jnp.select([k_true], [jnp.array([[[[InjSpec.kernel_x(X, X_out, E, 0.01, Ep) for Ep in E_grid] for E in E_grid] for X_out in X_grid] for X in X_grid])])
S_0 = jnp.array([S0X(0.01) for S0X in S_0f])
S_cont = jnp.array([[SCX(E, 0.01) for E in E_grid] for SCX in S_contf])

--> 703     dt = np.result_type(x)
    704   except TypeError as err:
    705     raise TypeError(f"Cannot determine dtype of {x}") from err

ValueError: invalid shape in fixed-type tuple.

In [28]:
from linx.injected_spec import set_spectra

In [ ]:

N_X = len(R)
NE = len(E_grid)

dy = jnp.log(E_grid[-1]/Emin)/(NE-1)

#create the grid to store the spectrums
#F_grid = jnp.zeros((N_X, NE)) 

#calculate the last row, which is important for trapezoidal rule
FX_E0 = jnp.array([S_cont[X,-1]/R[X,-1] + jnp.sum(K[X,:,-1,-1]*S_0[:]/(R[:,-1]*R[X,-1])) for X in range(N_X)])

#F_grid = set_spectra(F_grid, -1, FX_E0)
#i  = (NE - 1) - 1 # start at the second to last index, NE-2

def cond_func(i_vec):
    return i_vec[0] >= 0

#@partial(jax.jit, static_argnums=(0,))
def body_func(i_vec):
#while i >= 0: # Counting down
    i = i_vec[0]
    B = jnp.zeros( (N_X, N_X) )
    a = jnp.zeros( (N_X,   ) )

    I = jnp.identity(N_X)
    # Calculate the matrix B and the vector a
    for X in jnp.arange(N_X):
        # Calculate B, : <--> Xp
        B = B.at[X,:].set(-.5*dy*E_grid[i]*K[X,:,i,i] + R[X,i]*I[X,:])

        # Calculate a
        a = a.at[X].set(S_cont[X,i])
        for Xp in jnp.arange(N_X):
            a = a.at[X].add(K[X,Xp,i,-1]*S_0[Xp]/R[Xp,-1] + .5*dy*E_grid[-1]*K[X,Xp,i,-1]*i_vec[1][Xp,-1])
            for j in range(i, NE-1): # Goes from i+1 to NE-2
                a = a.at[X].add(dy*E_grid[j]*K[X,Xp,i,j]*i_vec[1][Xp,j])

    # Solve the system of linear equations of the form BF = a
    F_grid = set_spectra(i_vec[1], i,jnp.linalg.solve(B, a))

    return (i-1, F_grid)

F_grid = jax.lax.while_loop(cond_func, body_func, ((NE - 1) - 1, set_spectra(jnp.zeros((N_X, NE)), -1, FX_E0)))[1]

approx_zero = 1e-200
# Remove potential zeros
F_grid = F_grid.reshape( N_X*NE )
F_grid = F_grid.at[:].set(jnp.where(F_grid > approx_zero, F_grid, approx_zero))
#for i, f in enumerate(F_grid):
    #if f < approx_zero:
    #    F_grid[i] = approx_zero
#    F_grid.at[i].set(f < approx_zero, f, approx_zero)
F_grid = F_grid.reshape( (N_X, NE) )

# Define the output array...
sol = jnp.zeros( (N_X+1, NE) )
# ...and fill it
sol = sol.at[0     , :].set(E_grid)
sol = sol.at[1:N_X+1, :].set(F_grid)

sol


In [ ]:
body_func((NE - 1, set_spectra(jnp.zeros((N_X, NE)), -1, FX_E0)))

In [ ]:
jnp.arange(1, 3)

In [158]:
#@partial(jax.jit, static_argnums=0)
@jax.jit
def test_func(i):
    l = i[0] 
    for j in jnp.arange(l, 3):
        t = E_grid[j]
    return t


In [ ]:
test_func((0, E_grid))

In [ ]:
%timeit InjSpec.inverse_compton_kernel_photon(5, 0.01, 8)

In [ ]:
%timeit InjSpec.inverse_compton_kernel_photon(5, 0.01, 8)

In [23]:
print("Photon Rates Test")
print("---------------")
print("double photon", InjSpec.dphoton_pair_prod_rate(5, 0.01))
print("photonphoton", InjSpec.photon_photon_scattering_rate(5, 0.01))
print("BH", InjSpec.bethe_heitler_pair_prod_rate(1, 0.01), InjSpec.bethe_heitler_pair_prod_rate(3, 0.01), InjSpec.bethe_heitler_pair_prod_rate(5, 0.01))
print("CS", InjSpec.compton_scattering_rate(5, 0.01))
print("total", InjSpec.total_rate_photon(5, 0.01))
print()
print("Photon Kernel Test")
print("---------------")
print("photonphoton", InjSpec.photon_photon_scattering_kernel(5, 0.01, 8))
print("CS", InjSpec.compton_scattering_kernel_photon(5, 0.01, 8))
print("Inverse CS", InjSpec.inverse_compton_kernel_photon(5, 0.01, 8))
print("total", InjSpec.total_kernel_photon(5, 0.01, 8, 0), InjSpec.total_kernel_photon(5, 0.01, 8, 1), InjSpec.total_kernel_photon(5, 0.01, 8, 2))

Photon Rates Test
---------------
double photon 1.7273806782478688e-12
photonphoton 9.528579442514193e-18
BH 0.0 1.93931715860427e-22 4.226659078975712e-22
CS 2.7877735519401764e-20


ValueError: vmap was requested to map its argument along axis 0, which implies that its rank should be at least 1, but is only 0 (its shape is ())

In [ ]:
print("Electron Rates Test")
print("---------------")
print("Inverse CS", InjSpec.inverse_compton_rate(5, 0.01))
print("total", InjSpec.total_lepton_rate(5, 0.01))
print()
print("Electron Kernels Test")
print("---------------")
print("Double photon", InjSpec.dphoton_pair_prod_kernel(5, 0.01, 8))
print("BH", InjSpec.bethe_heitler_pair_prod_kernel(5, 0.01, 8))
print("CS", InjSpec.compton_scattering_kernel_electron(5, 0.01, 8))
print("Inverse CS", InjSpec.inverse_compton_kernel_lepton(5, 0.01, 8))
print("total", InjSpec.total_kernel_electron(5, 0.01, 8, 0), InjSpec.total_kernel_electron(5, 0.01, 8, 1), InjSpec.total_kernel_electron(5, 0.01, 8, 2))

In [ ]:
print("Positron Rates Test")
print("---------------")
print("Inverse CS", InjSpec.inverse_compton_rate(5, 0.01))
print("total", InjSpec.total_lepton_rate(5, 0.01))
print()
print("Positron Kernels Test")
print("---------------")
print("Double photon", InjSpec.dphoton_pair_prod_kernel(5, 0.01, 8))
print("BH", InjSpec.bethe_heitler_pair_prod_kernel(5, 0.01, 8))
print("CS", InjSpec.compton_scattering_kernel_electron(5, 0.01, 8))
print("Inverse CS", InjSpec.inverse_compton_kernel_lepton(5, 0.01, 8))
print("total", InjSpec.total_kernel_positron(5, 0.01, 8, 0), InjSpec.total_kernel_positron(5, 0.01, 8, 1), InjSpec.total_kernel_positron(5, 0.01, 8, 2))

In [28]:
print("Photon Rates Test NEW")
print("---------------")
print("double photon")
%timeit InjSpec.dphoton_pair_prod_rate(5, 0.01)
print("photonphoton")
%timeit InjSpec.photon_photon_scattering_rate(5, 0.01)
print("BH")
%timeit InjSpec.bethe_heitler_pair_prod_rate(1, 0.01)
print("CS")
%timeit InjSpec.compton_scattering_rate(5, 0.01)
print("total")
%timeit InjSpec.total_rate_photon(5, 0.01)
print()
print("Photon Kernel Test")
print("---------------")
print("photonphoton")
%timeit InjSpec.photon_photon_scattering_kernel(5, 0.01, 8)
print("CS")
%timeit InjSpec.compton_scattering_kernel_photon(5, 0.01, 8)
print("Inverse CS")
%timeit InjSpec.inverse_compton_kernel_photon(5, 0.01, 8)
print("total")
%timeit InjSpec.total_kernel_photon(5, 0.01, 8, 0)

Photon Rates Test NEW
---------------
double photon
31.2 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
photonphoton
507 μs ± 8.55 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
BH
337 μs ± 101 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CS
229 μs ± 1.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
total
15.4 ms ± 335 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Photon Kernel Test
---------------
photonphoton
247 μs ± 8.41 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CS
252 μs ± 9.08 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Inverse CS
315 μs ± 7.15 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
total
326 μs ± 8.11 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [32]:
print("Electron Rates Test NEW")
print("---------------")
print("Inverse CS")
%timeit InjSpec.inverse_compton_rate(5, 0.01)
print("total")
%timeit InjSpec.total_lepton_rate(5, 0.01)
print()
print("Electron Kernels Test")
print("---------------")
print("Double photon")
%timeit InjSpec.dphoton_pair_prod_kernel(5, 0.01, 8)
print("BH")
%timeit InjSpec.bethe_heitler_pair_prod_kernel(5, 0.01, 8)
print("CS")
%timeit InjSpec.compton_scattering_kernel_electron(5, 0.01, 8)
print("Inverse CS")
%timeit InjSpec.inverse_compton_kernel_lepton(5, 0.01, 8)
print("total")
%timeit InjSpec.total_kernel_electron(5, 0.01, 8, 0)

Electron Rates Test NEW
---------------
Inverse CS
17.2 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
total
17.1 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Electron Kernels Test
---------------
Double photon
377 μs ± 115 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
BH
230 μs ± 1.55 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
CS
231 μs ± 1.13 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Inverse CS
381 μs ± 109 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
total
453 μs ± 108 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
print("Electron Rates Test")
print("---------------")
print("Inverse CS")
%timeit InjSpec.inverse_compton_rate(5, 0.01)
print("total")
%timeit InjSpec.total_lepton_rate(5, 0.01)
print()
print("Electron Kernels Test")
print("---------------")
print("Double photon")
%timeit InjSpec.dphoton_pair_prod_kernel(5, 0.01, 8)
print("BH")
%timeit InjSpec.bethe_heitler_pair_prod_kernel(5, 0.01, 8)
print("CS")
%timeit InjSpec.compton_scattering_kernel_electron(5, 0.01, 8)
print("Inverse CS")
%timeit InjSpec.inverse_compton_kernel_lepton(5, 0.01, 8)
print("total")
%timeit InjSpec.total_kernel_electron(5, 0.01, 8, 0)

In [ ]:
print("Positron Rates Test")
print("---------------")
print("Inverse CS")
%timeit InjSpec.inverse_compton_rate(5, 0.01)
print("total")
%timeit InjSpec.total_lepton_rate(5, 0.01)
print()
print("Positron Kernels Test")
print("---------------")
print("Double photon")
%timeit InjSpec.dphoton_pair_prod_kernel(5, 0.01, 8)
print("BH")
%timeit InjSpec.bethe_heitler_pair_prod_kernel(5, 0.01, 8)
print("CS")
%timeit InjSpec.compton_scattering_kernel_electron(5, 0.01, 8)
print("Inverse CS")
%timeit InjSpec.inverse_compton_kernel_lepton(5, 0.01, 8)
print("total")
%timeit InjSpec.total_kernel_positron(5, 0.01, 8, 0)

In [ ]:
%timeit InjSpec.dphoton_pair_prod_rate(5, 0.01)

In [ ]:
InjSpec.dphoton_pair_prod_rate(5, 0.01)

In [ ]:
a = jnp.linspace(1, 10, 10)
a = jnp.where(a > 5, 0, a)
print(a)

In [ ]:
from scipy.integrate import dblquad, quad
from scipy.special import erf, jv


def h(t, z):
    return f(t) * g(z)


def f(t):
    return 0.5 * t * (erf(t - a) - 1) * jv(0, q * t)


def g(z):
    return math.exp(-((z - a) ** 2)/(2 * (s ** 2)))


def h1(z):
    return integral_of_f(z) * g(z)

def integral_of_f(z):
    return quad(f, 0, 2 * z)[0]  # here abserr is discarded


if __name__ == '__main__':
    a, q, s = 0, 2, 3  # set the constants

    result, abserr = dblquad(h, 0, 60, lambda z: 0, lambda z: 2 * z)
    print(f'result: {result}, abserr: {abserr}')

    result1 = quad(h1, 0, 60)[0]  # abserr here for result1 is not valid since abserr is already discarded in the function integral_of_f
    print(f'result1: {result1}')


In [11]:
@jax.jit
def f_thermal_photon_spec(ep, T):
    return (ep**2)/jnp.pi**2 * 1/(jnp.exp(ep/T) - 1)
#
@jax.jit
def dp_integrand(s, ep, T):
    s = jnp.exp(s)
    ep = jnp.exp(ep)
    b = jnp.sqrt(1 - (4*me**2)/s)
    dp_cross_section = jnp.pi * aFS**2 /(2*me**2) * (1-b**2) * ((3-b**4) * jnp.log((1+b)/(1-b)) - 2*b *(2-b**2))  
    return f_thermal_photon_spec(ep, T)/ep**2 * dp_cross_section * s *ep *s  

@jax.jit
def dphoton_pair_prod_rate(E, T):

    #Check if incoming energy is greater than me^2/(22T)
    #acropolis uses 50T for a smaller threshold
    """    if (type(E) != float) and (type(E) != int):
        E = jnp.where(E < (me**2)/(50*T), 0, E)
    else:
    """
    #if E < (me**2)/(50*T):
    #    return 0
    
    @jax.jit
    def inside_int(s):
        s = jnp.exp(s)
        b = jnp.sqrt(1 - (4*me**2)/s)
        dp_cross_section = jnp.pi * aFS**2 /(2*me**2) * (1-b**2) * ((3-b**4) * jnp.log((1+b)/(1-b)) - 2*b *(2-b**2))  
        return dp_cross_section * s *s  
    
    @jax.jit
    def integral_of_inside(ep):
        return quadgk(inside_int, [jnp.log(4*me**2), jnp.log(4*E *ep)], epsrel = eps/10, epsabs=0)[0]
    
    @jax.jit
    def outside_integral(ep, T):
        ep = jnp.exp(ep)
        return integral_of_inside(ep) * f_thermal_photon_spec(ep, T)/ep**2 *ep



    ep_ll = jnp.log((me**2)/E)
    ep_ul = jnp.log(Ephb_T_max*T)

    #I_dp_pp = dblquad(dp_integrand, ep_ll, ep_ul, lambda x: jnp.log(4*me**2), lambda x: jnp.log(4*E) + x, epsrel = eps, epsabs=0, args=(T,)) 
    I_dp_pp = quadgk(outside_integral, [ep_ll, ep_ul], epsrel = eps, epsabs=0, args=(T,))
    return 1/(8*E**2) * I_dp_pp[0]

In [97]:
@jax.jit
def icre_integrand(E_ph, ep, E, T):
    #equation B.23 in Hufnagel 2018
    F = F_func(E_ph, E, ep)
    return f_thermal_photon_spec(ep, T)/ep * F
@jax.jit
def F_func(E, Ep, ep):   
        """Function used for calculating rates and kernels related to inverse compton scattering 

        Parameters
        ----------
        E: float 
            Energy of outgoing particle (MeV)
        Ep: float 
            E', energy of incoming particle (MeV)
        ep: float
            epsilon, energy of thermal bath

        Returns
        -------
        float 
            F(E, E', epsilon), which is (dimensionless)
        """
        
        
        #should never happen if limits are right
        '''
        if not (ep <= E <= (4*ep*Ep**2)/(me**2 + 4*ep*Ep)):
            return 0
        '''
        
        G_ep = 4*ep*Ep/(me**2)
        q = E/(G_ep*(Ep-E))
        F = 2*q*jnp.log(q) + (1+2*q)*(1-q) + G_ep**2 * q**2 * (1 - q)/(2+2*G_ep*q)

        return F

@jax.jit
def inverse_compton_rate(E, T):
        """ 
        Inverse Compton Scattering Rate Eq. B.24 in Hufnagel 2018
        
        Parameters
        ----------
        E: float
            Energy of outgoing particle (MeV)
        T: float
            Temperature of the thermal bath (MeV)

        Returns
        -------
        float
            Gamma^IC_e+- (MeV)
            
        """
        @jax.jit
        def inside_int(E_ph, E, ep):
            return F_func(E_ph, E, ep)
        
        @jax.jit
        def integral_of_inside(ep, E):
            return quadgk(inside_int, [ep, 4*ep*E**2 /(me**2 + 4*ep*E)], epsrel = eps/10, epsabs=0, args = (E, ep))[0]

        @jax.jit
        def outside_integral(ep, T, E):
            return integral_of_inside(ep, E) * f_thermal_photon_spec(ep, T)/ep
        
        ep_ll = 0
        ep_ul = jnp.minimum(E - (me**2)/(4*E), Ephb_T_max*T)
        #E_ph_ll = lambda x: x
        #E_ph_ul = lambda x: 4*x*E**2 /(me**2 + 4*x*E)

        #return 2*jnp.pi*aFS**2 /E**2 * dblquad(icre_integrand, ep_ll, ep_ul, E_ph_ll, E_ph_ul,  epsrel=eps, epsabs=0, args=(E, T))[0]
        return 2*jnp.pi*aFS**2 /E**2 * quadgk(outside_integral, [ep_ll, ep_ul], epsrel=eps, epsabs=0, args=(T, E))[0]

#@jax.jit
def inverse_compton_rate_vmap(E, T):
        """ 
        Inverse Compton Scattering Rate Eq. B.24 in Hufnagel 2018
        
        Parameters
        ----------
        E: float
            Energy of outgoing particle (MeV)
        T: float
            Temperature of the thermal bath (MeV)

        Returns
        -------
        float
            Gamma^IC_e+- (MeV)
            
        """
        @jax.jit
        def inside_int(E_ph, E, ep):
            return F_func(E_ph, E, ep)
        
        @jax.jit
        def integral_of_inside(ep, E):
            I_i =  jax.vmap(quadgk, in_axes=(None, 0, 1, None, None, None))(inside_int, [ep, 4*ep*E**2 /(me**2 + 4*ep*E)], jnp.array((E, ep)), False, 0,  eps)
            return I_i[0]
        @jax.jit
        def outside_integral(ep, T, E):
            return integral_of_inside(ep, E) * f_thermal_photon_spec(ep, T)/ep
        
        
        ep_ul = jnp.minimum(E - (me**2)/(4*E), Ephb_T_max*T)
        ep_ll = jnp.zeros_like(ep_ul)
        #E_ph_ll = lambda x: x
        #E_ph_ul = lambda x: 4*x*E**2 /(me**2 + 4*x*E)

        #return 2*jnp.pi*aFS**2 /E**2 * dblquad(icre_integrand, ep_ll, ep_ul, E_ph_ll, E_ph_ul,  epsrel=eps, epsabs=0, args=(E, T))[0]
        I =  2*jnp.pi*aFS**2 /E**2 * jax.vmap(quadgk, in_axes=(None, 0, 0, None, None, None))(outside_integral, [ep_ll, ep_ul], jnp.array((T, E)), False, 0, eps)
        return I[0]

In [98]:
def printing(a, b, c, d, e):
    return (a, b, c, d, e)

In [ ]:
jax.vmap(printing, in_axes=[0, 1, None, None, 0])(jnp.arange(5), jnp.zeros((2, 5)), 0, 0, jnp.arange(5))

In [ ]:
import time
test_energies = jnp.linspace(1, 10, 10)
inverse_compton_rate_vmap(test_energies, jnp.full_like(test_energies, 0.01))
start = time.time()
print("hello")
end = time.time()
print(end - start)

In [9]:
@jax.jit
def loopfunc(E_list):
    for E_i in E_list:
        print(inverse_compton_rate(E_i, 0.01))

In [ ]:
start = time.time()
loopfunc(test_energies)
end = time.time()
print("time", end - start)

In [16]:
X_grid = jnp.arange(3)
E_grid = jnp.logspace(jnp.log(Emin), jnp.log(5), 50, base=jnp.e)
x1, x2 = jnp.meshgrid(X_grid, E_grid, indexing='ij')



In [ ]:
x1.flatten()

In [ ]:
x2.flatten()

In [ ]:
jax.vmap(InjSpec.rate_x, in_axes=(0, 0, None))(x1.flatten(), x2.flatten(), 0.01) 

In [ ]:
jnp.array([[InjSpec.rate_x(X, E, 0.01) for E in E_grid] for X in X_grid])

In [118]:
ii, jj, kk, ll = jnp.meshgrid( X_grid, X_grid,E_grid, E_grid,indexing='ij')

In [160]:
a =  jax.vmap(InjSpec.kernel_x, in_axes=(0, 0, 0, None, 0))(ii.flatten(), jj.flatten(), kk.flatten(), 0.01, ll.flatten())
a = jnp.reshape(a, (3, 3, 10, 10))
a = jnp.nan_to_num(a)


In [161]:
b = jnp.array([[[[InjSpec.kernel_x(X, X_out, E, 0.01, Ep) for Ep in E_grid] for E in E_grid] for X_out in X_grid] for X in X_grid])
b = jnp.nan_to_num(b)

In [ ]:
%timeit jax.vmap(InjSpec.rate_x, in_axes=(0, 0, None))(x1.flatten(), x2.flatten(), 0.01) 

In [ ]:
inverse_compton_rate(test_energies, 0.01)

In [ ]:
start = time.time()
print(jax.vmap(inverse_compton_rate, in_axes = (0, None))(test_energies, 0.01))
end = time.time()
print("time", end - start)

In [ ]:
%timeit loopfunc(test_energies)

In [ ]:
%timeit print(jax.vmap(inverse_compton_rate, in_axes = (0, None))(test_energies, 0.01))

In [ ]:
dphoton_pair_prod_rate(5, 0.01)

In [ ]:
%timeit dphoton_pair_prod_rate(5, 0.01)

In [ ]:
%timeit dphoton_pair_prod_rate(5, 0.01)

In [ ]:
%timeit dphoton_pair_prod_rate(5, 0.01)

In [ ]:
def test_func(x, y, z):
    return x*y

dblquad(test_func, 0, 1, lambda x: 0, lambda x: 4.*x , args=(2, ))

In [85]:
from quadax import quadgk, quadcc, quadts, romberg

In [11]:
gaussian = lambda x: 1/jnp.sqrt(jnp.pi) * jnp.exp(-x**2)

In [ ]:
quadgk(gaussian, [0, 1])

In [ ]:
%timeit quadgk(gaussian, [0, 1])

In [ ]:
%timeit quadcc(gaussian, [0, 1])

In [ ]:
%timeit quadts(gaussian, [0, 1])

In [ ]:
%timeit quad(gaussian, 0, 1)

In [ ]:
%timeit romberg(gaussian, [0, 1])

In [ ]:
%timeit jc.scipy.integrate.romb(gaussian, 0, 1)

In [ ]:
inverse_compton_rate(3, 0.01)

In [ ]:
inverse_compton_rate(3, 0.01)

In [ ]:
inverse_compton_rate(3, 0.01)

In [35]:
def photon_photon_scattering_rate(E, T):

    exp_fac = jnp.exp(-E*T/me**2)

    return 15568*(jnp.pi**3)/(3189375) * aFS**4 * me * (E/me)**3 * (T/me)**6 * exp_fac


In [5]:
import math
import time

In [12]:
def _rate_photon_photon( E, T):
    #if E > me2/T:
    #    return 0.
    expf = math.exp( -E*T/me**2 )

    return 0.151348 * (aFS**4.) * me * (E/me)**3. * (T/me)**6. * expf

@jax.jit
def photon_photon_scattering_rate_jit(E, T):

    exp_fac = jnp.exp(-E*T/me**2)

    return 15568*(jnp.pi**3)/(3189375) * aFS**4 * me * (E/me)**3 * (T/me)**6 * exp_fac

@jax.jit
def photon_photon_scattering_rate_lax(E, T):

    exp_fac = jax.lax.exp(-E*T/me**2)

    return 15568*(jnp.pi**3)/(3189375) * aFS**4 * me * (E/me)**3 * (T/me)**6 * exp_fac


In [13]:
start = time.time()
#for i in range(10000):
photon_photon_scattering_rate_lax(jnp.full((100), 5), jnp.full((100), 0.01))
end = time.time()
print(end-start)

0.12168550491333008


In [19]:
start = time.time()
#for i in range(10000):
photon_photon_scattering_rate_jit(jnp.full((100), 5), jnp.full((100), 0.01))
end = time.time()
print(end-start)

0.04653573036193848


In [18]:
n = 1000
start = time.time()
#for i in range(10000):
jax.vmap(photon_photon_scattering_rate_jit, in_axes=(0, 0))(jnp.full((n), 5), jnp.full((n), 0.01))
end = time.time()
print(end-start)

0.0024123191833496094


In [17]:
start = time.time()
for i in range(n):
    _rate_photon_photon(5, 0.01)
end = time.time()
print(end-start)

0.0011630058288574219
